In [5]:
# %% imports and setting up
from os import chdir as setwd 
from os import getcwd as getcwd

# setting working directory
from config import file_dir
setwd(file_dir)
print("Current working directory: ", getcwd())

# import data science libraries and scripts
import pandas as pd
import numpy as np
import csv
import gurobipy as gp
from gurobipy import GRB

Current working directory:  C:\Users\jjaso\Documents\ds08_renewable_energy_scheduling


## Main optimise function 

In [6]:
def optimise(forecast_output, aemo): 
    
    """
    optimisation model to minimise the cost 

    Parameters
    ----------
    forecast_output : string
        A csv file that contains the forecasted energy consumption and 
        production for 6 buildings and 6 solar panels. There are 2880 
        time period.
        
    aemo : string
        A csv file containing the electricity price for the 2880 time period 
        of month November 2020.
        
    Returns
    -------
    energy_cost_result : list
        A list integers of the energy cost for 12 problem instance
        
    """
    
    
    # --------------- import dataset for iput --------------
    
    # forecast values
    forecast = []
    building_consumption = []
    solar_production = []

    # sample output from the 
    with open(forecast_output,'r') as f:
        reader = csv.reader(f)
        for row in reader:
            forecast.append(row)

    for i in range(0,6):
        building_consumption.append(forecast[i])

    for i in range(6,12):
        solar_production.append(forecast[i])

    # netload[i,t] = building consumption[i,t] - solar production[i,t] 
    # where i is building/solar number and t is time 
    netload = []

    for t in range(1,2881):
        sum_building_consumption = 0
        sum_solar_production = 0
        for i in range(6):
            sum_building_consumption += float(building_consumption[i][t])
            sum_solar_production += float(solar_production[i][t])
        netload.append(sum_building_consumption-sum_solar_production)


    # electricity price data
    electricity = pd.read_csv(aemo)
    electricity = electricity.loc[electricity.index.repeat(2)]
    electricity = electricity.reset_index()

    price_elec = electricity['RRP']


    # crate day of the week for each time stamp in November 2020
    # starts with Sunday followed by 4 weeks, and ends on Monday
    e = []
    for i in range(1,8):
        e += [i for _ in range(96)]

    dayofweek = [7 for _ in range(96)] + e*4 + [1 for _ in range(96)]

    # create time of day
    # consist of 96 t(s), start with index 0 ends with 95

    t = [i for i in range(96)]


    optimData = {'Base_load' : netload ,
                'price' : price_elec,
                'dayofweek' : dayofweek,
                'timeofday' : t*30
                }
    optimData = pd.DataFrame(optimData)
    base_load = netload
    
    
    # --------------- optimisation model --------------
    
    energy_cost_result = []
    
    files = [    
        "phase2_instance_small_0.txt",
        "phase2_instance_small_1.txt",
        "phase2_instance_small_2.txt",
        "phase2_instance_small_3.txt",
        "phase2_instance_small_4.txt",
        "phase2_instance_large_0.txt",
        "phase2_instance_large_1.txt",
        "phase2_instance_large_2.txt",
        "phase2_instance_large_3.txt",
        "phase2_instance_large_4.txt"    
            ]

    
    for filenum in range(len(files)) : 
    
        # --------------- read problem instance --------------

        with open('rawdata/' + files[filenum]) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]

        lines = [line.split() for line in lines]
        lines

        # first line of problem instance values
        ppoi = lines[0]
        building_count = ppoi[1]
        solar_count = ppoi[2]
        battery_count = ppoi[3]
        reccur_act_count = ppoi[4]
        once_act_count = ppoi[5]

        # get building data
        building_instance = lines[1:1+int(building_count)]
        names = ['instance_type','building_id','no_small_rooms','no_large_rooms']
        building = pd.DataFrame(building_instance, columns = names)

        # get solar data
        solar_instance = lines[1+int(building_count):1+int(building_count)+int(solar_count)]
        names = ['instance_type','solar_id','building_id']
        solar = pd.DataFrame(solar_instance, columns = names)

        # get battery data 
        battery_instance = []
        for i in range(len(lines)) : 
            if lines[i][0] == 'c' :
                battery_instance.append(lines[i])

        if len(battery_instance) > 0 : 
            names = ['instance_type','battery_id','building_id','capacity_kwh','max_power_kwh','efficiency']
            battery_instance = pd.DataFrame(battery_instance, columns = names)

        battery = battery_instance

        # get recurring activity data
        recurring_activity_instance = []

        for i in range(len(lines)) : 
            if lines[i][0] == 'r' :
                if lines[i][6] == '0':
                    recurring_activity_instance.append(lines[i].copy() + [[]])
                else : 
                    recurring_activity_instance.append(lines[i][:7].copy() + [lines[i][7:].copy()])

        for i in range(len(recurring_activity_instance)): 
            names = ['instance_type','activity_id','no_of_rooms','room_size','load_kwh','duration','no_of_precendence','precendence_list']
            recurring_activity_instance = pd.DataFrame(recurring_activity_instance, columns = names)

        recurring_activity_ins = recurring_activity_instance

        # get once-off activity data
        once_activity_instance = []

        for i in range(len(lines)) : 
            if lines[i][0] == 'a' :
                if lines[i][8] == '0':
                    once_activity_instance.append(lines[i].copy() + [[]])
                else : 
                    once_activity_instance.append(lines[i][:9].copy() + [lines[i][9:].copy()])

        for i in range(len(recurring_activity_instance)): 
            names = ['instance_type','activity_id','no_of_rooms','room_size','load_kwh','duration','value','penalty','no_of_precendence','precendence_list']
            once_activity_instance = pd.DataFrame(once_activity_instance, columns = names)

        once_activity_ins = once_activity_instance

        building.iloc[:,1:] = building.iloc[:,1:].astype(int)
        building

        solar.iloc[:,1:] = solar.iloc[:,1:].astype(int)
        solar

        battery.iloc[:,1:] = battery.iloc[:,1:].astype(float)
        battery


        recurring_activity_ins.iloc[:,[1,2,4,5,6]] = recurring_activity_ins.iloc[:,[1,2,4,5,6]].astype(int)

        x = recurring_activity_ins.iloc[:,3]
        small_room = [0]*len(x)
        large_room = [0]*len(x)
        for i in range(len(x)):
            if x[i] == "S":
                small_room[i] = recurring_activity_ins.iloc[:,2][i]
            else: 
                large_room[i] = recurring_activity_ins.iloc[:,2][i]
        recurring_activity_ins['no_small_rooms'] = small_room
        recurring_activity_ins['no_large_rooms'] = large_room

        # --------------- set values for optimisation model --------------
        nbulding = 6
        nsolar = 6
        nperiod = 2880
        nbattery = battery_instance.shape[0]
        nreccur = recurring_activity_ins.shape[0]
        nonce = once_activity_ins.shape[0]
        nsmallroom = sum(building['no_small_rooms'])
        nlargeroom = sum(building['no_large_rooms'])

        w = 672
        s = 132
        d = 96

        # valid_t includes all the valid time 
        # week1 valid time 
        valid_t = list(range(s,s+32)) + list(range(s+d,s+d+32)) + list(range(s+2*d,s+2*d+32)) + list(range(s+3*d,s+3*d+32)) + list(range(s+4*d,s+4*d+32))

        # add week 2 valid time
        for i in range(7,12):
            valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 

        # add week 3 valid time
        for i in range(14,19):
            valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 

        # add week 4 valid time
        for i in range(21,26):
            valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 

        # add week 5 valid time
        i = 28
        valid_t = valid_t + list(range(s+i*d,s+i*d+32)) 

        # non valid time
        nonvalid_t = set(list(range(nperiod))) - set(valid_t)
        nonvalid_t = list(nonvalid_t)

        # there are 5 weeks in Nov 2020 that contains weekdays
        # list all the week time range 
        week1_t = list(range(d,d+w))
        week2_t = list(range(d+w,d+w*2))
        week3_t = list(range(d+w*2,d+w*3))
        week4_t = list(range(d+w*3,d+w*4))
        week5_t = list(range(d+w*4,2880))
        
        # --------------- optimisation for objective maxpower  --------------
        
        print("------------------------------------------------------------------------------------")
        print("objective maxpower optimisation for " + files[filenum] + " start")
        print("------------------------------------------------------------------------------------")
        
        m = gp.Model("MonashScheduling")
        m.setParam('TimeLimit', 3*60)
        

        ## variables
        net_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0,  name = 'net power') # net power at each time t 
        act_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'act power') # power from acts at each time t
        bat_level = m.addVars(nbattery, nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'bat level') # battery level status at each time t
        ract_begin = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, name = 'recurring activity begin') # 1 if activity begins 
        ract_ongoing = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, lb = 0, name = 'recurring activity ongoing') # 1 if activity is ongoing
        bat_charge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery charge')
        bat_discharge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery discharge')

        maxpower = m.addVar(name = "max power") # max power


        ## constraints

        # -- battery -- 


        for bat in range(nbattery):

            # each battery at start of the month is full
            m.addConstr(bat_level[bat,0] == battery['capacity_kwh'][bat])

            # each battery has to be less than the capacity
            for t in range(nperiod):
                m.addConstr(bat_level[bat,t] <= battery['capacity_kwh'][bat])

        # -- activity --

        for act in range(nreccur):

            # each activity happend only once a week 
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week1_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week2_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week3_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week4_t) == 1)

            # precendence activity must happend before 
            if len(recurring_activity_ins['precendence_list'][act]) > 0 : 
                prec_list = recurring_activity_ins['precendence_list'][act]
                for prec_act in prec_list: 
                    prec_act = int(prec_act)
                    m.addConstr(gp.quicksum(ract_begin[act, t] * t for t in week1_t) >= gp.quicksum(ract_begin[prec_act, t] * t for t in week1_t))

            # same time every week 
            for num in range(768, nperiod):
                m.addConstr(ract_begin[act, num] == ract_begin[act, num - 672])
                m.addConstr(ract_ongoing[act, num] == ract_ongoing[act, num - 672])

            # activity should last as long as their period 
            for t in range(nperiod-recurring_activity_ins['duration'][act]):
                for dur in range(recurring_activity_ins['duration'][act]):
                    m.addConstr(ract_begin[act,t] <= ract_ongoing[act, t + dur])

            # activity is only between monday to friday 9am-5pm
            for t in nonvalid_t:
                m.addConstr(ract_ongoing[act,t] == 0)
                m.addConstr(ract_begin[act,t] == 0)


        for t in range(nperiod):
            # no of small room use in time t for all acitivity is less than equal to number of small rooms in all buildings
            m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_small_rooms'][act] for act in range(nreccur))
                        <= nsmallroom )

            # no of large room use in time t for all acitivity is less than equal to number of large rooms in all buildings
            m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_large_rooms'][act] for act in range(nreccur))
                        <= nlargeroom )


        # -- max power -- 
        m.addConstr(maxpower == gp.max_(net_power[t] for t in range(nperiod)))


        # -- power calculation --

        # power from all activity that is happening at each time t 
        for t in range(nperiod):
             m.addConstr(act_power[t] == gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['load_kwh'][act] 
                                                    * (recurring_activity_ins['no_small_rooms'][act] + recurring_activity_ins['no_large_rooms'][act])
                                                    for act in range(nreccur)) )

        # -- battery level calculation -- 
        for t in range(1,nperiod):
            for bat in range(nbattery):
               m.addConstr(bat_level[bat,t] == bat_level[bat,t-1] + (bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5) 
                            - (bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5))


        # -- net power calculation -- 
        for t in range(nperiod):
            m.addConstr(net_power[t] == base_load[t] + act_power[t] + gp.quicksum(bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5 
                           - bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5 for bat in range(nbattery)))


        ## objective
        m.setObjective(maxpower)


        ## optimize 
        m = m.presolve()
        m = m.relax()
        m.optimize()

        obj = m.getObjective()
        max_limit = obj.getValue()
        
        print("------------------------------------------------------------------------------------")
        print("objective maxpower optimisation for " + files[filenum] + " finished")
        print("------------------------------------------------------------------------------------")
        
        # --------------- optimisation for the main objective function  --------------
        
        print("------------------------------------------------------------------------------------")
        print("Main objective function optimisation for " + files[filenum] + " start")
        print("------------------------------------------------------------------------------------")
        
        m = gp.Model("MonashScheduling")
        m.setParam('TimeLimit', 3*60)

        ## variables
        net_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0,  name = 'net power') # net power at each time t 
        act_power = m.addVars(nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'act power') # power from acts at each time t
        bat_level = m.addVars(nbattery, nperiod, vtype = GRB.CONTINUOUS, lb = 0, name = 'bat level') # battery level status at each time t
        ract_begin = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, name = 'recurring activity begin') # 1 if activity begins 
        ract_ongoing = m.addVars(nreccur, nperiod, vtype = GRB.BINARY, lb = 0, name = 'recurring activity ongoing') # 1 if activity is ongoing
        bat_charge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery charge')
        bat_discharge = m.addVars(nbattery, nperiod, vtype = GRB.BINARY, name = 'battery discharge')

        maxpower = m.addVar(name = "max power") # max power


        ## constraints

        # -- battery -- 


        for bat in range(nbattery):

            # each battery at start of the month is full
            m.addConstr(bat_level[bat,0] == battery['capacity_kwh'][bat])

            # each battery has to be less than the capacity
            for t in range(nperiod):
                m.addConstr(bat_level[bat,t] <= battery['capacity_kwh'][bat])

        # -- activity --

        for act in range(nreccur):

            # each activity happend only once a week 
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week1_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week2_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week3_t) == 1)
            m.addConstr(gp.quicksum(ract_begin[act,t] for t in week4_t) == 1)

            # precendence activity must happend before 
            if len(recurring_activity_ins['precendence_list'][act]) > 0 : 
                prec_list = recurring_activity_ins['precendence_list'][act]
                for prec_act in prec_list: 
                    prec_act = int(prec_act)
                    m.addConstr(gp.quicksum(ract_begin[act, t] * t for t in week1_t) >= gp.quicksum(ract_begin[prec_act, t] * t for t in week1_t))

            # same time every week 
            for num in range(768, nperiod):
                m.addConstr(ract_begin[act, num] == ract_begin[act, num - 672])
                m.addConstr(ract_ongoing[act, num] == ract_ongoing[act, num - 672])

            # activity should last as long as their period 
            for t in range(nperiod-recurring_activity_ins['duration'][act]):
                for dur in range(recurring_activity_ins['duration'][act]):
                    m.addConstr(ract_begin[act,t] <= ract_ongoing[act, t + dur])

            # activity is only between monday to friday 9am-5pm
            for t in nonvalid_t:
                m.addConstr(ract_ongoing[act,t] == 0)
                m.addConstr(ract_begin[act,t] == 0)


        for t in range(nperiod):
            # no of small room use in time t for all acitivity is less than equal to number of small rooms in all buildings
            m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_small_rooms'][act] for act in range(nreccur))
                        <= nsmallroom )

            # no of large room use in time t for all acitivity is less than equal to number of large rooms in all buildings
            m.addConstr(gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['no_large_rooms'][act] for act in range(nreccur))
                        <= nlargeroom )


        # -- max power -- 
        m.addConstr(maxpower == gp.max_(net_power[t] for t in range(nperiod)))


        # -- power calculation --

        # power from all activity that is happening at each time t 
        for t in range(nperiod):
             m.addConstr(act_power[t] == gp.quicksum(ract_ongoing[act,t] * recurring_activity_ins['load_kwh'][act] 
                                                    * (recurring_activity_ins['no_small_rooms'][act] + recurring_activity_ins['no_large_rooms'][act])
                                                    for act in range(nreccur)) )

        # -- battery level calculation -- 
        for t in range(1,nperiod):
            for bat in range(nbattery):
               m.addConstr(bat_level[bat,t] == bat_level[bat,t-1] + (bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5) 
                            - (bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5))


        # -- net power calculation -- 
        for t in range(nperiod):
            m.addConstr(net_power[t] == base_load[t] + act_power[t] + gp.quicksum(bat_charge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** -0.5 
                           - bat_discharge[bat,t] * battery['max_power_kwh'][bat] * battery['efficiency'][bat] ** 0.5 for bat in range(nbattery)))


        ## objective

        #m.setObjective(maxpower)
        #m.setObjective(gp.quicksum(net_power[t]*price_elec[t] for t in range(nperiod)), GRB.MINIMIZE)
        m.setObjective(gp.quicksum(((0.25)*net_power[t]*price_elec[t])/1000 for t in range(nperiod)) + 0.005*max_limit**2)


        ## optimize 
        m.optimize()
        
        obj = m.getObjective()
        energy_cost_result.append(obj.getValue())
        
        print("------------------------------------------------------------------------------------")
        print("Main objective function optimisation for " + files[filenum] + " finished")
        print("------------------------------------------------------------------------------------")
        
        
        
        
    return energy_cost_result



In [7]:
result = optimise('forecasting/saved_forecasts/forecast_naive.csv','rawdata/PRICE_AND_DEMAND_202011_VIC1.csv')

------------------------------------------------------------------------------------
objective maxpower optimisation for phase2_instance_small_0.txt start
------------------------------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-26
Set parameter TimeLimit to value 180
Presolve removed 1290639 rows and 278887 columns
Presolve time: 1.59s
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 15231 rows, 32154 columns and 106039 nonzeros
Model fingerprint: 0xbe2f40fe
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 12 rows and 5571 columns
Presolve time: 0.07s
Presolved: 

 10664  6512 26231.2628  429  232 26508.8343 26225.4592  1.07%  24.8   95s
H11260  6509                    26430.271141 26225.4592  0.77%  21.1   95s
H13394  8039                    26423.755336 26225.4592  0.75%  18.8   99s
 13394  5806 26230.1387  272  189 26423.7553 26225.4592  0.75%  20.9  100s
H15728  5857                    26415.068506 26225.4609  0.72%  16.3  101s
 16548  6870 26225.7443  129 1343 26415.0685 26225.4609  0.72%  17.1  108s
 16741  7803 26225.7685  134 1331 26415.0685 26225.4609  0.72%  17.0  113s
 17683  7815 26225.8876  162 1301 26415.0685 26225.4609  0.72%  16.5  119s
 17695  9107 26225.9604  162 1299 26415.0685 26225.4609  0.72%  16.5  121s
H19007  9107                    26405.303737 26225.4609  0.68%  14.7  121s
 19065  8346 26226.0174   31  469 26405.3037 26226.0126  0.68%  17.5  129s
 19067  8349 26226.0204   32  516 26405.3037 26226.0163  0.68%  17.5  130s
H21657  9494                    26403.798347 26226.0220  0.67%  15.0  134s
 21657 10626 26226.6409  

     0     0 24930.3013    0 1734          - 24930.3013      -     -   10s
     0     0 24930.3040    0 1728          - 24930.3040      -     -   10s
     0     0 24934.3652    0 1739          - 24934.3652      -     -   11s
     0     0 24934.8787    0 1757          - 24934.8787      -     -   11s
     0     0 24934.9673    0 1729          - 24934.9673      -     -   11s
     0     0 24934.9872    0 1731          - 24934.9872      -     -   11s
     0     0 24937.5708    0 1788          - 24937.5708      -     -   11s
     0     0 24937.8238    0 1850          - 24937.8238      -     -   12s
     0     0 24937.8485    0 1851          - 24937.8485      -     -   12s
     0     0 24939.0315    0 1791          - 24939.0315      -     -   12s
     0     0 24939.0753    0 1821          - 24939.0753      -     -   12s
     0     0 24939.0859    0 1845          - 24939.0859      -     -   12s
     0     0 24939.5032    0 1881          - 24939.5032      -     -   13s
     0     0 24939.6506  

Optimal objective  1.078865010e+03
------------------------------------------------------------------------------------
objective maxpower optimisation for phase2_instance_small_2.txt finished
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
Main objective function optimisation for phase2_instance_small_2.txt start
------------------------------------------------------------------------------------
Set parameter TimeLimit to value 180
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1274257 rows, 311041 columns and 2897524 nonzeros
Model fingerprint: 0x45425a1b
Model has 1 general constraint
Variable types: 11521 continuous, 299520 integer (299520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e-05, 7e-02]
  Bounds range   

   5   3.06549596e+03 -5.81351584e+03  1.43e+01 1.67e-16  1.59e-01     0s
   6   2.03276932e+03 -2.21891346e+03  7.29e+00 1.28e-16  7.56e-02     0s
   7   1.54817908e+03 -4.08437971e+02  4.21e+00 7.58e-16  3.52e-02     0s
   8   1.32162652e+03  3.05708986e+02  2.68e+00 2.91e-16  1.89e-02     0s
   9   1.18935347e+03  7.16204386e+02  1.27e+00 6.57e-16  8.89e-03     0s
  10   1.11761004e+03  9.36523863e+02  4.18e-01 1.80e-16  3.37e-03     0s
  11   1.09330361e+03  1.03085674e+03  1.21e-01 1.30e-16  1.14e-03     0s
  12   1.08356669e+03  1.07777686e+03  1.34e-03 3.81e-16  9.39e-05     0s
  13   1.08319670e+03  1.08319114e+03  5.10e-07 1.11e-16  8.93e-08     0s
  14   1.08319626e+03  1.08319626e+03  1.04e-11 3.65e-15  8.93e-11     0s

Barrier solved model in 14 iterations and 0.40 seconds (0.28 work units)
Optimal objective 1.08319626e+03

Crossover log...

    4586 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00              

------------------------------------------------------------------------------------
objective maxpower optimisation for phase2_instance_small_4.txt start
------------------------------------------------------------------------------------
Set parameter TimeLimit to value 180
Presolve removed 1201640 rows and 278730 columns
Presolve time: 1.34s
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 15216 rows, 32311 columns and 103100 nonzeros
Model fingerprint: 0xcc37a1b7
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 12 rows and 5571 columns
Presolve time: 0.08s
Presolved: 15204 rows, 26740 columns, 95978 nonzeros

Ordering time: 0.03s

Barrier statistics:
 Dense

H15368  4957                    25090.624757 24863.3352  0.91%  16.3  132s
 15368  4904 24864.1153   89 1292 25090.6248 24863.3352  0.91%  17.5  136s
 17420  6611 24865.7723  175 1185 25090.6248 24863.3352  0.91%  16.3  141s
H18400  7246                    25089.879387 24863.3352  0.90%  14.7  144s
 18400  9127 24863.3460  129  436 25089.8794 24863.3352  0.90%  16.1  147s
H20907  9654                    25089.852431 24863.3352  0.90%  13.6  149s
 20907 10189 24865.9206  212 1134 25089.8524 24863.3352  0.90%  14.5  152s
 22613 12323 24866.2423  273 1053 25089.8524 24863.3352  0.90%  13.8  155s
 24035 13546 24866.3502  302 1024 25089.8524 24863.3352  0.90%  13.3  162s
H24851 13546                    25089.848567 24863.3352  0.90%  12.3  162s
 26051 15776 24863.6768  250  386 25089.8486 24863.3352  0.90%  12.9  165s
H28893 17135                    25089.763710 24863.3352  0.90%  11.3  167s
 29740 17962 24866.5239  362 1000 25089.7637 24863.3352  0.90%  11.8  177s
 29752 18404 24866.5690  

     0     0 25705.2834    0 2890          - 25705.2834      -     -   63s
     0     0 25705.7562    0 2908          - 25705.7562      -     -   64s
     0     0 25705.8123    0 3311          - 25705.8123      -     -   64s
     0     0 25705.8201    0 3366          - 25705.8201      -     -   64s
     0     0 25707.5722    0 3195          - 25707.5722      -     -   66s
     0     0 25707.7869    0 3115          - 25707.7869      -     -   67s
     0     0 25707.7998    0 3092          - 25707.7998      -     -   68s
     0     0 25708.3464    0 2703          - 25708.3464      -     -   69s
     0     0 25708.4013    0 3202          - 25708.4013      -     -   70s
     0     0 25708.4097    0 3452          - 25708.4097      -     -   70s
     0     0 25708.6728    0 3318          - 25708.6728      -     -   72s
     0     0 25708.7250    0 3046          - 25708.7250      -     -   73s
     0     0 25708.7297    0 3341          - 25708.7297      -     -   74s
     0     0 25708.8865  

AttributeError: Unable to retrieve attribute 'x'